In [0]:
!pip install -U -q PyDrive

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
import tensorflow as tf
import tensorflow 

from tensorflow import keras

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
% cd /content/drive/My\ Drive/major_project/deep_defense

/content/drive/My Drive/major_project/deep_defense


In [0]:
from preprocess_dataset import standardised_X as X, dummy_Y as Y

Using TensorFlow backend.


In [0]:
import pandas as pd
import numpy as np

In [0]:
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Bidirectional
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

In [0]:
print(X.shape[0])
print(Y.shape)
print(len(X[1]))

500000
(500000, 5)
29


In [0]:
features = len(X[0])
samples = X.shape[0]
train_len = 50
input_len = samples - train_len
I = np.zeros((samples - train_len, train_len, features))

for i in range(input_len):
    temp = np.zeros((train_len, features))
    for j in range(i, i + train_len - 1):
        temp[j-i] = X[j]
    I[i] = temp

In [0]:
# output_len = samples - train_len
# output_features = Y.shape[1]
# output = np.zeros((samples - train_len, train_len, output_features))
# output.shape

# for i in range(output_len):
#     temp = np.zeros((train_len, output_features))
#     for j in range(i, i + train_len - 1):
#         temp[j-i] = Y[j]
#     output[i] = temp

In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(I, Y[50:500000], test_size = 0.4, shuffle=False)
# I = I.reindex(np.random.permutation(I.index))


In [0]:
def create_baseline():
    model = Sequential()
    
    model.add(Bidirectional(LSTM(64, activation='tanh', kernel_regularizer='l2')))
#     model.add(Bidirectional(LSTM(64, activation='tanh')))
#     model.add(Bidirectional(LSTM(64, activation='tanh')))
#     model.add(Bidirectional(LSTM(64, activation='tanh')))

    model.add(Dense(128, activation = 'relu', kernel_regularizer='l2'))
    model.add(Dense(5, activation = 'sigmoid', kernel_regularizer='l2'))
    
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    
    return model

In [0]:
model = create_baseline()

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
model.fit(X_train, Y_train, epochs = 1, verbose = 1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/1


In [0]:

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 128)               48128     
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 645       
Total params: 65,285
Trainable params: 65,285
Non-trainable params: 0
_________________________________________________________________


In [0]:
# model.predict(X_test)

In [0]:
# print(Y_test)

In [0]:
scores = model.evaluate(X_test, Y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

acc: 99.85%


In [0]:
model.save('brnn_model.h5')